# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [20]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.widgets import RunDetails

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.57.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [15]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'capstone_automl'
project_folder = './pipeline-project'

experiment=Experiment(ws, experiment_name)

In [3]:
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-271377
aml-quickstarts-271377
westus2
a24a24d5-8d87-4c8a-99b6-91ed2d2df51f


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [17]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
# amlcompute_cluster_name = "auto-ml"
amlcompute_cluster_name = "udacity-capstone-azureml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

InProgress...
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [12]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = True
key = "breast-cancer-test"
description_text = "Wisconsin Breast Cancer Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/calemolech/nd00333_capstone/refs/heads/main/data.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [23]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [24]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

Submitting remote run.
No run_configuration provided, running on udacity-capstone-azureml with default configuration
Running on remote compute: udacity-capstone-azureml


Experiment,Id,Type,Status,Details Page,Docs Page
capstone_automl,AutoML_ec5ecf44-7746-4d5b-8df3-543e12c3bc7f,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn mo

2025-01-12 04:24:54.826826: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 04:24:55.773204: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 04:24:56.053731: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 04:24:58.267971: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-12 04:25:03.492361: W tensorflow/compiler/tf2

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
capstone_automl,AutoML_ec5ecf44-7746-4d5b-8df3-543e12c3bc7f,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  In order to accurately evaluate the model(s) trained by AutoML, we leverage a dataset that the model is not trained on. Hence, if the user doesn't provide an explicit validation dataset, a part of the training dataset is used to achieve this. For smaller datasets (fewer than 20,000 samples), cross-validation is leveraged, else a single hold-out set is split from the training data to serve as the validation dataset. Hence, for your input data we leverage cross-validation with 10 folds, if the number of training samples are fewer than 1000, and 3 folds in all other cases.
              Learn more about cross validation: https://aka.ms/AutomatedMLCrossValidation
DETAILS:      
+------------------------------+
|Number of folds               |
+==============================+
|10                            |
+------

{'runId': 'AutoML_ec5ecf44-7746-4d5b-8df3-543e12c3bc7f',
 'target': 'udacity-capstone-azureml',
 'status': 'Completed',
 'startTimeUtc': '2025-01-12T03:54:10.280234Z',
 'endTimeUtc': '2025-01-12T04:24:16.181608Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'udacity-capstone-azureml',
  'AMLSettingsJsonString': '{"path":null,"name":"capstone_automl","subscription_id":"a24a24d5-8d87-4c8a-99b6-91ed2d2df51f","resource_group":"aml-quickstarts-271377","workspace_name":"quick-starts-ws-271377","region":"westus2","compute

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [25]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [26]:
best_run, best_model = remote_run.get_output()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
capstone_automl,AutoML_ec5ecf44-7746-4d5b-8df3-543e12c3bc7f_53,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [27]:
best_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/notebook271377/code/Users/odl_user_271377/nd00333_capstone')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassif...lassifier(max_features='log2', min_samples_leaf=0.01, min_samples_split=0.01, n_estimators=10, n_jobs=1))]))], flatten_transform=False, weights=[0.18181818181818182, 0.09090909090909091, 0.09090909090909091, 0.2727272727272727, 0.09090909090909091, 0.18181818181818182, 0.09090909090909091]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [28]:
metrics = best_run.get_metrics()
metrics['accuracy']

0.9841478696741854

In [29]:
#TODO: Save the best model
import joblib
from azureml.core.model import Model

description = "Best-fit AutoML model trained on Wisoconsin Breast Cancer dataset"

os.makedirs('outputs', exist_ok=True)
joblib.dump(best_model, filename="outputs/automl-model.pkl")
automl_model = remote_run.register_model(model_name='automl-breastcancer', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [30]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict Wisoconsin Breast Cancer diagnosis - Malignant(1) or Benign(0)',
    auth_enabled=True
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [34]:
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'automl-breastcancer')


myenv = best_run.get_environment()
entry_script = 'predict.py'
#best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='automl-breastcancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-01-12 04:47:48+00:00 Creating Container Registry if not exists.
2025-01-12 04:47:49+00:00 Use the existing image.
2025-01-12 04:47:51+00:00 Submitting deployment to compute.
2025-01-12 04:47:59+00:00 Checking the status of deployment automl-breastcancer..
2025-01-12 04:51:26+00:00 Checking the status of inference endpoint automl-breastcancer.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [35]:
service.update(enable_app_insights=True)

In [36]:
print("State "+ service.state)
print("Key " + service.get_keys()[0])
print("Swagger URI " + service.swagger_uri)
print("Scoring URI " + service.scoring_uri)

State Healthy
Key Z1QTfRMUKajcctfq1lDgC0oM6C0vHtng
Swagger URI http://38b0e7d9-38e8-4b4d-a11b-c8df15f9ca76.westus2.azurecontainer.io/swagger.json
Scoring URI http://38b0e7d9-38e8-4b4d-a11b-c8df15f9ca76.westus2.azurecontainer.io/score


In [37]:
%run test_endpoint.py

['M']


In [43]:
service.get_logs()

'2025-01-12T04:55:41,793509046+00:00 - rsyslog/run \n2025-01-12T04:55:41,797876549+00:00 - gunicorn/run \n2025-01-12T04:55:41,799390314+00:00 | gunicorn/run | \n2025-01-12T04:55:41,803668753+00:00 | gunicorn/run | ###############################################\n2025-01-12T04:55:41,806338472+00:00 | gunicorn/run | AzureML Container Runtime Information\n2025-01-12T04:55:41,807992929+00:00 | gunicorn/run | ###############################################\n2025-01-12T04:55:41,809302841+00:00 | gunicorn/run | \n2025-01-12T04:55:41,810511968+00:00 - nginx/run \n2025-01-12T04:55:41,812883551+00:00 | gunicorn/run | \n2025-01-12T04:55:41,818548990+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20240918.v1\n2025-01-12T04:55:41,819349027+00:00 | gunicorn/run | \n2025-01-12T04:55:41,820285534+00:00 | gunicorn/run | \n2025-01-12T04:55:41,821431098+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.


In [46]:
df_temp = pd.read_csv('./data.csv')

In [47]:
df_temp

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.00,0.12,0.28,0.30,0.15,...,17.33,184.60,2019.00,0.16,0.67,0.71,0.27,0.46,0.12,NaN
1,842517,M,20.57,17.77,132.90,1326.00,0.08,0.08,0.09,0.07,...,23.41,158.80,1956.00,0.12,0.19,0.24,0.19,0.28,0.09,NaN
2,84300903,M,19.69,21.25,130.00,1203.00,0.11,0.16,0.20,0.13,...,25.53,152.50,1709.00,0.14,0.42,0.45,0.24,0.36,0.09,NaN
3,84348301,M,11.42,20.38,77.58,386.10,0.14,0.28,0.24,0.11,...,26.50,98.87,567.70,0.21,0.87,0.69,0.26,0.66,0.17,NaN
4,84358402,M,20.29,14.34,135.10,1297.00,0.10,0.13,0.20,0.10,...,16.67,152.20,1575.00,0.14,0.20,0.40,0.16,0.24,0.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.00,0.11,0.12,0.24,0.14,...,26.40,166.10,2027.00,0.14,0.21,0.41,0.22,0.21,0.07,NaN
565,926682,M,20.13,28.25,131.20,1261.00,0.10,0.10,0.14,0.10,...,38.25,155.00,1731.00,0.12,0.19,0.32,0.16,0.26,0.07,NaN
566,926954,M,16.60,28.08,108.30,858.10,0.08,0.10,0.09,0.05,...,34.12,126.70,1124.00,0.11,0.31,0.34,0.14,0.22,0.08,NaN
567,927241,M,20.60,29.33,140.10,1265.00,0.12,0.28,0.35,0.15,...,39.42,184.60,1821.00,0.17,0.87,0.94,0.27,0.41,0.12,NaN
